In [1]:
from PIL import Image
import math
from fpdf import FPDF

def process_image(image_path, book_height, num_pages, first_page, orientation, strip_width):
    # Load the image
    image = Image.open(image_path)

    # Adjust the image orientation
    if orientation == 'horizontal':
        image = image.rotate(90, expand=True)

    # Resize the image to fit the book
    image_width = strip_width * num_pages
    image = image.resize((int(image_width), book_height))

    # Generate the pattern
    # This is a placeholder for the pattern generation logic
    # You would need to implement this based on how you want to transform the image into stripes

    # Save the pattern preview
    preview_path = 'pattern_preview.jpg'
    image.save(preview_path)

    return preview_path

def create_pdf(image_path, strip_width, num_pages):
    pdf = FPDF(orientation='P', unit='cm', format='A4')
    pdf.add_page()

    # Add the stripes to the PDF
    for page in range(num_pages):
        x_position = (page % 2) * strip_width  # Alternating positions for the stripes
        pdf.image(image_path, x=x_position, y=0, w=strip_width)

    # Save the PDF
    pdf_path = 'output.pdf'
    pdf.output(pdf_path)
    return pdf_path

# Example usage
image_path = 'SampleDog.jpeg'  # Replace with the path to the uploaded image
book_height = 20  # Example book height in cm
num_pages = 100  # Example number of pages
first_page = 1
orientation = 'vertical'  # 'vertical' or 'horizontal'
strip_width = 1.5  # Width of the strip in cm

preview_path = process_image(image_path, book_height, num_pages, first_page, orientation, strip_width)
pdf_path = create_pdf(preview_path, strip_width, num_pages)

print(f"Preview generated at: {preview_path}")
print(f"PDF generated at: {pdf_path}")


Preview generated at: pattern_preview.jpg
PDF generated at: output.pdf


In [9]:
import os
from PIL import Image
from fpdf import FPDF

def resize_and_stretch_image(image_path, book_height_cm, num_pages, strip_width_cm, resized_image_path):
    # Open the original image
    image = Image.open(image_path)

    # Calculate new dimensions
    new_height_px = int(book_height_cm * 37.795275591)  # Convert cm to pixels
    total_width_cm = num_pages * strip_width_cm
    new_width_px = int(total_width_cm * 37.795275591)  # Convert total width from cm to pixels

    # Resize and stretch the image
    resized_image = image.resize((new_width_px, new_height_px), Image.ANTIALIAS)

    # Save the resized and stretched image
    resized_image.save(resized_image_path)

    return resized_image

def create_pdf(resized_image, strip_width_cm, num_pages, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Constants for A4 size and margins
    A4_WIDTH_CM = 21
    MARGIN_CM = 1

    # Calculate the width of each strip in pixels
    strip_width_px = resized_image.width // num_pages

    pdf = FPDF(orientation='P', unit='cm', format='A4')
    pdf.add_page()  # Start by adding a new page
    x_position = MARGIN_CM
    y_position = MARGIN_CM

    # Iterate over each strip and add it to the PDF
    for strip in range(num_pages):
        if x_position + strip_width_cm > A4_WIDTH_CM - MARGIN_CM:
            pdf.add_page()
            x_position = MARGIN_CM

        # Calculate the bounding box for the strip
        left = strip * strip_width_px
        right = left + strip_width_px
        box = (left, 0, right, resized_image.height)

        # Cut the strip from the image and save
        strip_image = resized_image.crop(box)
        strip_path = os.path.join(output_folder, f'strip_{strip}.jpg')
        strip_image.save(strip_path)

        # Add the strip to the PDF
        pdf.image(strip_path, x=x_position, y=y_position, w=strip_width_cm)

        # Update the x_position for the next strip
        x_position += strip_width_cm + MARGIN_CM

    # Save the PDF
    pdf_path = os.path.join(output_folder, 'output.pdf')
    pdf.output(pdf_path)
    return pdf_path


# Example usage
image_path = 'SampleDog.jpeg'  # Replace with the path to the uploaded image
resized_image_path = 'resized_stretched_SampleDog.jpeg'  # Path to save the resized and stretched image
book_height_cm = 20  # Example book height in cm
num_pages = 100  # Example number of pages
strip_width_cm = 1.5  # Width of the strip in cm
output_folder = 'strips_output'  # Folder to save the strips

# Resize and stretch the image to match the book height and total width for all pages, then save it
resized_image = resize_and_stretch_image(image_path, book_height_cm, num_pages, strip_width_cm, resized_image_path)

# Create the PDF with the strips
pdf_path = create_pdf(resized_image, strip_width_cm, num_pages, output_folder)
print(f"Resized and stretched image saved at: {resized_image_path}")
print(f"PDF generated at: {pdf_path}")


/var/folders/c2/x_cgj8b10s38q85qqx_c88hr0000gn/T/ipykernel_35771/3223188132.py:15: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  resized_image = image.resize((new_width_px, new_height_px), Image.ANTIALIAS)


Resized and stretched image saved at: resized_stretched_SampleDog.jpeg
PDF generated at: strips_output/output.pdf
